# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span>

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/1_air_quality_feature_backfill.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data
2. Connect to the Hopsworks feature store
3. Create feature groups and insert them to the feature store

![tutorial-flow](../../images/01_featuregroups.png)

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
!pip install geopy folium streamlit-folium --q

# Hosted notebook environments may not have the local features package
import os

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("⚙️ Downloading modules...")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/features/air_quality.py')
    os.system('wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/functions.py')
    os.system('wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/air_quality/target_cities.json')
    print('✅ Done!')
else:
    print("Local environment")

In [ ]:
import datetime
import time
import requests
import json

import pandas as pd
import folium

from features import air_quality
from functions import *

import warnings
warnings.filterwarnings("ignore")

---

## <span style='color:#ff5f27'> 🌍 Representing the Target cities </span>

In [ ]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

In [ ]:
# Create a folium map centered on the first location in the list
my_map = folium.Map(location=[42.57, -44.092], zoom_start=3)

for continent in target_cities:
        for city_name, coords in target_cities[continent].items():
            folium.CircleMarker(
                location=coords,
                popup=city_name
            ).add_to(my_map)

my_map

In [ ]:
# # Save the map to an HTML file
# my_map.save("map_all_target_cities.html")

## <span style='color:#ff5f27'> 🌫 Processing Air Quality data</span>

### [🇪🇺 EEA](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)
#### EEA means European Environmental Agency

In [ ]:
target_cities["EU"]

In [ ]:
df_eu = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_eu.csv")

In [ ]:
df_eu.isna().sum().sum()

In [ ]:
print("Size of this dataframe:", df_eu.shape)

df_eu.sample(3)

### [🇺🇸 USEPA](https://aqs.epa.gov/aqsweb/documents/data_api.html#daily)
#### USEPA means United States Environmental Protection Agency
[Manual downloading](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)



In [ ]:
target_cities["US"]

In [ ]:
df_us = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_us.csv")

In [ ]:
df_us.isna().sum().sum()

In [ ]:
print("Size of this dataframe:", df_us.shape)

df_us.sample(3)

### <span style="color:#ff5f27;">🏢 Processing special city - `Seattle`</span>
#### We need different stations across the Seattle. 
I downloaded daily `PM2.5` data manually [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [ ]:
target_cities["Seattle"]

In [ ]:
df_seattle = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_seattle.csv")

In [ ]:
df_seattle.isna().sum().sum()

In [ ]:
print("Size of this dataframe:", df_seattle.shape)

df_seattle.sample(3)

In [ ]:
df_seattle.city_name.value_counts()

### <span style="color:#ff5f27;">🌟 All together</span>

In [ ]:
df_air_quality = pd.concat([df_eu, df_us, df_seattle]).reset_index(drop=True)

In [ ]:
df_air_quality.sample(5)

In [ ]:
df_air_quality.shape

### <span style="color:#ff5f27;">🛠 Feature Engineering</span>

In [ ]:
df_air_quality.head(2)

In [ ]:
df_air_quality['date'] = pd.to_datetime(df_air_quality['date'])

In [ ]:
df_air_quality = air_quality.feature_engineer_aq(df_air_quality)
df_air_quality = df_air_quality.dropna()

In [ ]:
df_air_quality.isna().sum().sum()

In [ ]:
df_air_quality.shape

In [ ]:
df_air_quality.columns

---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

In [ ]:
df_weather = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_weather.csv")

In [ ]:
df_weather.city_name.value_counts()

In [ ]:
df_weather.sample(3)

---

In [ ]:
df_air_quality.date = pd.to_datetime(df_air_quality.date)
df_weather.date = pd.to_datetime(df_weather.date)

df_air_quality["unix_time"] = df_air_quality["date"].apply(convert_date_to_unix)
df_weather["unix_time"] = df_weather["date"].apply(convert_date_to_unix)

In [ ]:
df_air_quality.date = df_air_quality.date.astype(str)
df_weather.date = df_weather.date.astype(str)

---

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['unix_time','city_name'],
    online_enabled=False,
    event_time=["unix_time"]
)    

In [ ]:
air_quality_fg.insert(df_air_quality)

### <span style='color:#ff5f27'> 🌦 Weather Data

In [ ]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['unix_time','city_name'],
    online_enabled=False,
    event_time=["unix_time"]
) 

In [ ]:
weather_fg.insert(
    df_weather, 
    write_options={"wait_for_job": True},
)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Feature Pipeline 
 </span> 

In the following notebook you will parse data and insert it into Feature Groups.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/2_air_quality_feature_pipeline.ipynb)
